# Example for using SCD Transformer and Compute with ConDynS

We demonstrate here how to use SCD Transformer for writing SCDs with your custom prompts. Then, we show how to compute ConDynS.

In [ ]:
import os
from convokit import Corpus, download
from convokit.convo_similarity import SCD
from convokit.convo_similarity.condyns import ConDynS
from convokit.genai import GenAIConfigManager

In [ ]:
corpus = Corpus(filename=download("friends-corpus"))

## Write SCD and SoP with SCDWriter

In [ ]:
### Config your GenAI API keys
config = GenAIConfigManager()

In [ ]:
### Define your own formatter function for your data
def format_friends_transcript_from_convokit(convo):
    utt_lst = convo.get_utterance_ids()
    speaker_ids = {}
    transcript = ""
    for utt_id in utt_lst:
        utt = corpus.get_utterance(utt_id)
        if "TRANSCRIPT_NOTE" not in utt.speaker.id:
            if utt.speaker.id not in speaker_ids:
                speaker_ids[utt.speaker.id] = 1 + len(speaker_ids)
            transcript += "Speaker"+str(speaker_ids[utt.speaker.id]) + " : " + utt.text+ "\n\n"
    return transcript

In [ ]:
### Prepare your own prompt for writing the SCD with your data
friends_summary_prompt = """
Write a short summary capturing the trajectory of a casual conversation. 
Do not include specific topics, events, or arguments from the conversation. The style you should avoid is illustrated in 
Example Sentence 1: “Speaker1 said they had a difficult day at work, and mentioned that their boss was unfair. Speaker2 listened and agreed that bosses can be tough, then suggested they go out for dinner to forget about it..” Instead, you should include indicators of sentiments (e.g., warmth, empathy, humor, nostalgia, vulnerability, support), individual intentions (e.g., building rapport, offering reassurance, seeking validation, self-disclosure, active listening, gentle disagreement, creating distance), and conversational strategies (if any) such as “collaborative storytelling,” “inside jokes,” “mirroring emotions,” and “affectionate teasing.” 
The following sentences demonstrate the style you should follow: 
Example Sentence 2: “Both speakers have similar feelings and appeared mutually supportive. Speaker1 initiates with a moment of self-disclosure, and Speaker2 responds with empathy and validation. Both speakers build on this exchange, strengthening their rapport.” 
Example Sentence 3: “The two speakers connected with back-and-forth affectionate teasing. Throughout the conversation, they kept building on each other's humor with playful remarks, creating a lighthearted and comfortable discussion.” Overall, the trajectory summary should capture the key moments where the emotional connection of the conversation notably changes. Here is an example of a complete trajectory summary: The conversation begins with two speakers exchanging neutral, surface-level comments. Speaker1 then shifts the tone by sharing a personal anecdote, prompting Speaker2 to respond with warmth and empathy. Speaker1 elaborates on their story and their need, but Speaker2 does not extend their support but retracts it. 
Now, provide the trajectory summary for the following conversation. 
Conversation Transcript: {transcript}. 
Now, summarize this conversation. Remember, do not include specific topics, claims, or arguments from the conversation. Instead, try to capture the speakers' sentiments, intentions, and conversational/persuasive strategies. Limit the trajectory summary to 80 words. 
Trajectory Summary:"""

Initialize your SCD transformer

In [ ]:
MODEL_PROVIDER = "gemini"

In [ ]:
scd_transformer = SCD(
        model_provider=MODEL_PROVIDER,
        config=config,
        custom_scd_prompt=friends_summary_prompt,
        custom_prompt_dir="friends_prompts",
        generate_scd=True,
        generate_sop=True,
        scd_metadata_name="machine_scd",
        sop_metadata_name="machine_sop",
        conversation_formatter=format_friends_transcript_from_convokit
    )

In [ ]:
conversation_ids = list(corpus.get_conversation_ids())[:2]
selector = lambda conv: conv.id in conversation_ids

In [ ]:
corpus = scd_transformer.transform(corpus, selector=selector)

In [ ]:
convo = corpus.get_conversation(conversation_ids[0])
print("SCD: ", convo.meta["machine_scd"])
print("SoP: ", convo.meta["machine_sop"])

## Compute ConDynS Score


In [ ]:
condyns = ConDynS(model_provider=MODEL_PROVIDER, config=config)

In [ ]:
convo_id1 = conversation_ids[0]
convo_id2 = conversation_ids[1]
convo1 = corpus.get_conversation(convo_id1)
convo2 = corpus.get_conversation(convo_id2)

transcript1 = "\n\n".join(format_friends_transcript_from_convokit(corpus, convo_id1))
transcript2 = "\n\n".join(format_friends_transcript_from_convokit(corpus, convo_id2))
sop1 = convo1.meta["machine_sop"]
sop2 = convo2.meta["machine_sop"]

result = condyns.compute_bidirectional_similarity(transcript1, transcript2, sop1, sop2)
score = condyns.compute_score_from_results(result)
print("ConDynS Result: ", result)
print("\n\nConDynS Score: ", score)